<a href="https://colab.research.google.com/github/pacozaa/data-playground-notebook/blob/main/DataSetToJSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
!pip install nltk datasets

In [74]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
# Download the Punkt tokenizer data (only needed once)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [75]:
from nltk.tokenize import word_tokenize

def combine_words(text, x):
    split_words = word_tokenize(text)
    combined_words = []

    # Iterate over the split words in steps of x
    for i in range(0, len(split_words), x):
        # Combine x words into a single string
        combined_word = ' '.join(split_words[i:i+x])
        combined_words.append(combined_word)

    return combined_words

def combine_words_from_split_words(split_words, x):
    combined_words = []

    # Iterate over the split words in steps of x
    for i in range(0, len(split_words), x):
        # Combine x words into a single string
        combined_word = ' '.join(split_words[i:i+x])
        combined_words.append(combined_word)

    return combined_words

# Example usage
text = "This is a sample text to demonstrate the combination of words."
x = 3  # Number of words to combine
combined_words = combine_words(text, x)
print(combined_words)

['This is a', 'sample text to', 'demonstrate the combination', 'of words .']


In [76]:
def saveToFile(name,dataset_name,data):
  dataWithId= [
      {"content": item, "id": str(uuid.uuid4())}
      for item in data
  ]
  json_array = json.dumps(dataWithId, indent=2)
  file_name = f"""{name}-{len(dataWithId)}-{dataset_name.replace("/", "-")}.json"""
  with open(file_name, "w") as f:
      f.write(json_array)

In [77]:
import json
import uuid
import random
def extract_sentences(dataset,sample_size,dataset_name,column_name):
  # Initialize an empty list to store the sentences
  sentences = []
  # Initialize an empty list to store the words
  words = []

  # Iterate over the dataset and split the text into sentences
  for example in dataset["train"].shuffle().select(range(200)):
      text = example[column_name]
      # SENTENCE
      # Split the text into sentences
      split_sentences = sent_tokenize(text)
      # Extend the list with the split sentences
      sentences.extend(split_sentences)

      # WORD
      # Split the text into words
      split_words = word_tokenize(text)
      combined_words = combine_words_from_split_words(split_words, 5)
      # Extend the list with the split words
      words.extend(combined_words)

  #Filter words
  words=[word for word in words if any(c.isalnum() for c in word)]



  # Shuffle the sentences and words list
  random.shuffle(sentences)
  random.shuffle(words)
  mix=sentences+words
  random.shuffle(mix)
  # Truncate the sentences list to the first 50 sentences
  sentences = sentences[:sample_size]
  words = words[:sample_size]
  mix = mix[:sample_size]

  # Optionally, save the JSON array to a file
  saveToFile("sentence",dataset_name,sentences)
  saveToFile("word",dataset_name,words)
  saveToFile("mix",dataset_name,mix)
  return sentences, words, mix

In [78]:
from datasets import load_dataset
dataset_list = [
    {
        "name": "imdb",
        "column": "text"
    },
    {
        "name": "sentence-transformers/stsb",
        "column": "sentence1"
    },
    {
        "name": "roneneldan/TinyStories",
        "column": "text"
    },
    # {
    #     "name": "derek-thomas/ScienceQA",
    #     "column": "question"
    # },
    {
        "name": "derek-thomas/dataset-creator-reddit-bestofredditorupdates",
        "column": "content"
    },
    {
        "name":"LIDIA-HESSEN/vencortex-BusinessNewsDataset",
        "column":"text"
    },
    {
        "name":"yashraizad/yelp-open-dataset-business",
        "column":"categories"
    },
    {
        "name":"yashraizad/top-reviews-per-business",
        "column":"text"
    },
    {
        "name":"warrencain/Business_Knowledge_Dataset_Llama_3.1_Medium_Token_Pairs",
        "column":"Output"
    },
    {
        "name":"darrel999/business-java-code",
        "column":"output"
    }
]

all_sentences=[]
all_words=[]
all_mix=[]
sample_size=50
# Loop throught dataset_name_list and run extract_sentences(dataset)
for dataset_data in dataset_list:
  print(f"""Loading.....{dataset_data["name"]}""")
  # Load Dataset
  dataset_name=dataset_data["name"]
  dataset = load_dataset(dataset_name)
  print(f"""data length: {len(dataset["train"])}""")
  column_name=dataset_data["column"]
  # Print all the column names in the dataset
  # print(dataset["train"].column_names)

  sentences, words, mix = extract_sentences(dataset,sample_size,dataset_name,column_name)
  all_sentences = all_sentences + sentences
  all_words = all_words + words
  all_mix = all_mix + mix

random.shuffle(all_sentences)
random.shuffle(all_words)
random.shuffle(all_mix)
print(f"""Total sentences: {len(all_sentences)}""")
print(f"""Total words: {len(all_words)}""")
print(f"""Total mix: {len(all_mix)}""")
# Truncate the sentences list to the first 50 sentences
all_sentences = all_sentences[:50]
all_words = all_words[:150]
all_mix = all_mix[:100]
saveToFile("all_sentences","",all_sentences)
saveToFile("all_words","",all_words)
saveToFile("all_mix","",all_mix)

Loading.....imdb
data length: 25000
Loading.....sentence-transformers/stsb
data length: 5749
Loading.....roneneldan/TinyStories
data length: 2119719
Loading.....derek-thomas/dataset-creator-reddit-bestofredditorupdates
data length: 11595
Loading.....LIDIA-HESSEN/vencortex-BusinessNewsDataset
data length: 469361
Loading.....yashraizad/yelp-open-dataset-business
data length: 150346
Loading.....yashraizad/top-reviews-per-business
data length: 53340
Loading.....warrencain/Business_Knowledge_Dataset_Llama_3.1_Medium_Token_Pairs
data length: 29362
Loading.....darrel999/business-java-code
data length: 53183
Total sentences: 450
Total words: 450
Total mix: 450
